In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, DecimalType
from pyspark.sql.functions import regexp_replace
import pandas as pd

In [13]:
#environment building
spark = SparkSession.builder.appName("Pyspark app").config("spark.some.option", "some-value").getOrCreate()

In [14]:
#charge data
df_features = spark.read.option('header', True).option('delimiter', '|').option('inferSchema', True).csv('../Assets/features.csv')
df_historic_sales = spark.read.option('header', True).option('delimiter', '|').csv('../Assets/historic_sales.csv')

df_store_info = spark.read.option('header', True).option('delimiter', ';').option('inferSchema', True).csv('../Assets/store_info.csv')

In [15]:
#main join 
df_Store_info_Sales = df_store_info.join(df_historic_sales, df_store_info['Store'] == df_historic_sales['Store'], 'left').select(df_store_info['Store'], df_store_info['Store'], df_store_info['Size'], 
                                                                                                                                df_historic_sales['Dept'], df_historic_sales['Date'], df_historic_sales['Weekly_Sales'],
                                                                                                                                df_historic_sales['IsHoliday'])

In [41]:
#Second semester filter
df_2012Sales = df_Store_info_Sales.select(df_Store_info_Sales['Store'], df_Store_info_Sales['Store'], df_Store_info_Sales['Size'], 
                                            df_Store_info_Sales['Dept'], df_Store_info_Sales['Date'], df_Store_info_Sales['Weekly_Sales'],
                                            df_Store_info_Sales['IsHoliday']).filter(df_Store_info_Sales['Date'].between('2012-07-01', '2012-12-31'))

In [44]:
df_2012Sales = df_2012Sales.withColumn('Weekly_Sales', regexp_replace('Weekly_Sales', ',', '.').cast(DecimalType()))#convert of Weekly_Sales colun to Decimal
df_2012SalesAmounts = df_2012Sales.groupBy('Store').sum('Weekly_Sales')#givin a totaml amount of sales 
df_2012SalesAmounts = df_2012SalesAmounts.orderBy(df_2012SalesAmounts['sum(Weekly_Sales)'].desc()) #sorting data

In [16]:
df_Store_info_Sales = df_Store_info_Sales.dropna()#drop of rows with null
df_Store_info_Sales2 = df_Store_info_Sales.withColumn('Weekly_Sales', regexp_replace('Weekly_Sales', ',', '.').cast(DecimalType()))#convert of Weekly_Sales colun to Decimal

In [28]:
df_totalSales = df_Store_info_Sales2.groupBy('Store').sum('Weekly_Sales')#givin a totaml amount of sales 
df_totalSales2 = df_totalSales.orderBy(df_totalSales['sum(Weekly_Sales)'].desc()) #sorting data

In [30]:
df_totalSalesCount = df_Store_info_Sales2.groupBy('Store').count()#count of sales per store

In [22]:
df_maxvalues = df_Store_info_Sales.groupBy('Store').max('Size')#getting size of stores
df_maxvalues = df_maxvalues.orderBy(df_maxvalues['max(Size)'].desc())#sorting data

In [25]:
df_biggerStores = df_maxvalues.join(df_totalSales2, df_maxvalues['Store'] == df_totalSales2['Store'], 'inner').select(df_maxvalues['Store'], df_maxvalues['max(Size)'], df_totalSales2['sum(Weekly_Sales)'])#union of dataframes total sales with max size

In [39]:
df_biggerStores_SalesCount_Desc = df_biggerStores.join(df_totalSalesCount, df_biggerStores['Store'] == df_totalSalesCount['Store'], 'inner').select(df_biggerStores['Store'], df_biggerStores['max(Size)'], df_biggerStores['sum(Weekly_Sales)'], df_totalSalesCount['count']).orderBy(df_biggerStores['max(Size)'].desc())#sorting data desc
df_biggerStores_SalesCount_Asce = df_biggerStores.join(df_totalSalesCount, df_biggerStores['Store'] == df_totalSalesCount['Store'], 'inner').select(df_biggerStores['Store'], df_biggerStores['max(Size)'], df_biggerStores['sum(Weekly_Sales)'], df_totalSalesCount['count']).orderBy(df_totalSalesCount['count'].asc())#sorting data asc

In [23]:
df_historic_salesCount = df_historic_sales.groupBy('Store').count()#count of sales